# Mental Health in the Tech Industry Data Processing

Let's create a class for gathering the data, statistical calculation and processing data for presentation and visualization.

In [1]:
import sqlite3
import numpy as np

In [37]:
class DataProcessing:
    ''' Class for gathering the data, statistical calculation and processing data. '''
    
    def __init__(self, path, db_name):
        self.path = path
        self.db_name = db_name
    
    def get_table(self, table):
        ''' Get table content from the database. '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute(f"SELECT * FROM {table}")
        table = c.fetchall()
        conn.close()
        return table

    def get_all_answers_per_q(self, q_id):
        ''' Get all unique answers from Answer table for particular question 
            represented by QuestionID number from the Question table. '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute("SELECT AnswerText FROM Answer WHERE QuestionID = ?", (q_id,))
#         answers = set(c.fetchall())
        answers = list(dict.fromkeys(c.fetchall())) # let's keep the order
        conn.close()
        return answers
    
    def get_all_years_per_q(self, q_id):
        ''' Get all unique years from Answer table for particular question 
            represented by QuestionID. '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute("SELECT SurveyID FROM Answer WHERE QuestionID = ?", (q_id,))
        years = list(dict.fromkeys(c.fetchall()))
        conn.close()
        return years
    
    def get_users_no_per_q(self, q_id):
        ''' Get number of all answers for the question. '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute("SELECT count(UserID) FROM Answer WHERE QuestionID = ?", (q_id,))
        user_no = c.fetchone()[0]
        conn.close()
        return user_no
    
    def get_users_no_for_q_and_answer(self, q_id, answer):
        ''' Get number of users from Answer table for particular question number (QuestionID) and answer (AnswerText). '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute("SELECT count(UserID) FROM Answer WHERE QuestionID = ? and AnswerText = ?", (q_id, answer))
        user_no = c.fetchone()[0]
        conn.close()
        return user_no
    
    def get_users_no_per_answer(self, q_id):
        ''' Get frequency of the answers for particular question number (QuestionID) in Answer table. '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute("SELECT AnswerText, count(UserID) FROM Answer WHERE QuestionID = ? GROUP BY AnswerText", (q_id,))
        user_no = c.fetchall()
        conn.close()
        return user_no
    
    def get_table_based_value_from_column(self, table, column, value):
        ''' Get particular answer from provided table. '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute(f"SELECT * FROM {table} WHERE {column} = ?", (value,))
        value = c.fetchall()
        conn.close()
        return value

    def get_answers_for_q_less_occ(self, q_id, qty):
        ''' Get answers for provided question q_id where occurency is less than qty. '''
        q_id, qty = int(q_id), int(qty)
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute(f'SELECT AnswerText, count(UserID) as UNo FROM Answer WHERE QuestionID = ? GROUP BY AnswerText HAVING UNo < ?', 
                  (q_id, qty))
        answers = c.fetchall()
        conn.close()
        return answers

    def get_answers_for_q_greater_occ(self, q_id, qty):
        ''' Get answers for provided question q_id where occurency is greater than qty. '''
        q_id, qty = int(q_id), int(qty)
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute(f'SELECT AnswerText, count(UserID) as UNo FROM Answer WHERE QuestionID = ? GROUP BY AnswerText HAVING UNo > ?', 
                  (q_id, qty))
        answers = c.fetchall()
        conn.close()
        return answers
    
    def get_answers_for_q_in_year(self, q_id, year):
        ''' Get all answers from Answer table for particular question (QuestionID)
            in given year (SurveyID). '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute("SELECT AnswerText FROM Answer WHERE QuestionID = ? and SurveyID = ?", (q_id, year))
        answers = set(c.fetchall())
        conn.close()
        return answers
    
    def get_users_no_for_q_and_answer_in_year(self, q_id, answer, year):
        ''' Get number of users from Answer table for particular question number (QuestionID) and answer (AnswerText)
            in given year (SurveyID). '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute("SELECT count(UserID) FROM Answer WHERE QuestionID = ? and AnswerText = ? and SurveyID = ?", (q_id, answer, year))
        user_no = c.fetchone()[0]
        conn.close()
        return user_no
    
    def get_users_no_per_q_in_year(self, q_id, year):
        ''' Get number of all answers for the question in given year (SurveyID). '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute("SELECT count(UserID) FROM Answer WHERE QuestionID = ? and SurveyID = ?", (q_id, year))
        user_no = c.fetchone()[0]
        conn.close()
        return user_no
    
    def get_answers_for_question(self, q_id):
        ''' Get answers content for given question. '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute(f"SELECT AnswerText FROM Answer WHERE QuestionID = ?", (q_id,))
        table = c.fetchall()
        conn.close()
        return table
    
    def get_answers_for_questions(self, q_ids):
        ''' Get answers content for given questions. '''
        values_no = ('?, '*len(q_ids))[:-2]
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute(f"SELECT AnswerText FROM Answer WHERE QuestionID = {values_no}", (*q_ids))
        table = c.fetchall()
        conn.close()
        return table
    
    def get_table_for_questions(self, q_ids):
        ''' Get table content for given questions. '''
        values_no = ('?, '*len(q_ids))[:-2]
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute(f"SELECT * FROM Answer WHERE QuestionID = {values_no}", (*q_ids))
        table = c.fetchall()
        conn.close()
        return table
    
    def get_different_answers_for_2q(self, q_id1, q_id2):
        ''' Get different answers on two questions for the same user. '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute(f"""SELECT AnswerText, UserID as uid FROM Answer 
        WHERE QuestionID = ? and AnswerText != (SELECT AnswerText FROM Answer WHERE UserID = uid and QuestionID = ?)""",
                 (q_id1, q_id2))
        table = c.fetchall()
        conn.close()
        return table
    
    def get_different_answers_for_2q_v2(self, q_id1, q_id2):
        ''' Get different answers on two questions for the same user.
            Much faster than get_different_answers_for_2q. '''
        tab_q1 = self.get_answers_ordered_by_uid(q_id1)
        tab_q2 = self.get_answers_ordered_by_uid(q_id2)
        diff_table = self._get_diff_table(tab_q1, tab_q2)
        return self._get_combined_tab_uid_based(diff_table, tab_q1)
        
    def get_answers_ordered_by_uid(self, q_id):
        ''' Get the answers ordered by UserID for given question. '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute(f"SELECT AnswerText, UserID FROM Answer WHERE QuestionID = ? Order by UserID", (q_id,))
        table = c.fetchall()
        conn.close()
        return table
    
    @staticmethod
    def _get_diff_table(tab1, tab2):
        ''' Returns the difference between two lists. '''
        table1 = set(tab1)
        table2 = set(tab2)
        return table2.difference(table1)
    
    @staticmethod
    def _get_combined_tab_uid_based(tab1, tab2):
        ''' Returns a list which combine first values from lists of tuples tab1 and tab2
            based on the second values.
            tab1 and tab2 need to have values of tuples of length 2. '''
        result_list = []
        for wc, wuid in list(tab1):
            for lc, luid in list(tab2):
                if luid == wuid:
                    result_list.append((lc, wc, wuid))
        return result_list    
    
    def get_users_no_per_two_q(self, q_id1, q_id2):
        ''' Get number of users who answered for provided two questions. '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute(f"""SELECT count(UserID), UserID as uid 
        FROM Answer WHERE QuestionID = ? and 
        EXISTS(SELECT AnswerText FROM Answer WHERE UserID = uid and QuestionID = ?)""",
                 (q_id1, q_id2))
        user_no = c.fetchone()[0]
        conn.close()
        return user_no
    
    def get_all_answers_based_on_answer_and_q(self, q_id, answer):
        ''' Get all answers for all users which answered on given question (q_id) in particular way (answer). '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute(f"""SELECT * FROM Answer 
                  WHERE UserID in (SELECT UserID FROM Answer WHERE AnswerText = ? and QuestionID = ?) 
                  Order by UserID""", (answer, q_id))
        table = c.fetchall()
        conn.close()
        return table
    
    def get_some_answers_based_on_answer_and_q(self, q_ids, q_id, answer):
        ''' Get answers on q_ids questions for all users which answered on given question (q_id) in particular way (answer). '''
        values_no = ('?, '*len(q_ids))[:-2]
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute(f"""SELECT * FROM Answer 
                  WHERE QuestionID in ({values_no}) 
                  and UserID in (SELECT UserID FROM Answer WHERE AnswerText = ? and QuestionID = ?) 
                  Order by UserID""", (*q_ids, answer, q_id))
        table = c.fetchall()
        conn.close()
        return table
    
    def get_some_answer_no_based_on_answer_and_q(self, q_id1, q_id2, answer):
        ''' Get number of answers on q_id1 question for all users 
            which answered on given question (q_id2) in particular way (answer). '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute(f"""SELECT DISTINCT AnswerText, Count(UserID) OVER (PARTITION BY AnswerText) FROM Answer 
                  WHERE QuestionID = ?
                  and UserID in (SELECT UserID FROM Answer WHERE AnswerText = ? and QuestionID = ?)""", 
                  (q_id1, answer, q_id2))
        table = c.fetchall()
        conn.close()
        return table
    
    def get_answers_distribution_for_q(self, q_id):
        ''' Get users percentage per answer for given question. '''
        answers = self.get_users_no_per_answer(q_id)
        return self._change_total_to_pct(answers)      
        
    def _change_total_to_pct(self, tab):
        ''' Change input table of answers with total occurrence to the percentage representation.
            Only table (answer,..., count) is supported. '''
#         sum_of_answers = sum([no for _, no in tab]) # works only for (answer, count)
        sum_of_answers = sum([row[-1] for row in tab]) # works for (answer,..., count) 
        table = []
#         for answer, value in tab: # works only for (answer, count) 
#             table.append((answer, self._round_2(100*value/sum_of_answers)))
        for row in tab: # works for (answer,..., count) 
            answer, value = row[0], row[-1]
            if len(row) > 2:
                table.append((answer, *row[1:-1], self._round_2(100*value/sum_of_answers)))
            else:
                table.append((answer, self._round_2(100*value/sum_of_answers)))
        return table
    
    def get_answers_distribution_for_q_by_years(self, q_id):
        ''' Get users percentage per answer for given question. '''
        answers = self.get_users_no_per_answer_by_years(q_id)
        return self._change_total_to_pct_yearly(q_id, answers)      
    
    def get_users_no_per_answer_by_years(self, q_id):
        ''' Get frequency of the answers along the years for particular question number (QuestionID) in Answer table. '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute("""SELECT AnswerText, SurveyID, count(UserID) 
        FROM Answer WHERE QuestionID = ? 
        GROUP BY SurveyID, AnswerText""", (q_id,))
        user_no = c.fetchall()
        conn.close()
        return user_no
    
    def _change_total_to_pct_yearly(self, q_id, tab):
        ''' Change input table of answers with total occurrence to the percentage representation.
            The calculation are done year by year.
            Only table (answer, year, ..., count) is supported. '''
        table = []
        years = [year[0] for year in self.get_all_years_per_q(q_id)]
        for year in years:
            answers_no = [row[-1] for row in tab if row[1] == year] 
            for row in tab[:len(answers_no)]:
                answer, value = row[0], row[-1]
                if len(row) > 2:
                    table.append((answer, *row[1:-1], self._round_2(100*value/sum(answers_no))))
                else:
                    table.append((answer, self._round_2(100*value/sum(answers_no))))
            del tab[:len(answers_no)]
        return table
        
    @staticmethod
    def _round_2(x):
        return round(x , 2)
    
    def get_avg_for_quantitative_q(self, q_id):
        ''' Get average for given quantitative question (q_id). '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute(f'SELECT avg(AnswerText) FROM Answer WHERE QuestionID = ?', (q_id,))
        avg = c.fetchone()[0]
        conn.close()
        return self._round_2(avg)
    
    def get_hist_for_quantitative_q(self, q_id):
        answers = self.get_answers_for_question(q_id)
        return self._get_hist(answers)
    
    @staticmethod
    def _get_hist(answers):
        data = [int(answer[0]) for answer in answers]
        return np.histogram(data)
    
    def get_question_text(self, q_id):
        ''' Get given question text. '''
        conn = sqlite3.connect(f'{self.path}{self.db_name}.sqlite')
        c = conn.cursor()
        c.execute(f'SELECT questiontext FROM Question WHERE questionid = ?', (q_id,))
        q_text = c.fetchone()[0]
        conn.close()
        return q_text

In [38]:
data_collector = DataProcessing('DB/', 'mental_health')

In [76]:
data_collector.get_users_no_for_q_and_answer(2, 'Male')

3044

In [77]:
data_collector.get_users_no_per_answer(2)

[('Female', 1024),
 ('Male', 3044),
 ('Non-binary', 13),
 ('Other', 107),
 ('n/a', 24),
 ('non-binary', 6)]

In [78]:
data_collector.get_users_no_per_q(2)

4218

In [79]:
data_collector.get_table_based_value_from_column('Answer', 'AnswerText', 'Idaho')

[('Idaho', 2014, 420, 4),
 ('Idaho', 2016, 1549, 4),
 ('Idaho', 2016, 2364, 4),
 ('Idaho', 2016, 2663, 4),
 ('Idaho', 2016, 2364, 51),
 ('Idaho', 2016, 2663, 51),
 ('Idaho', 2017, 2838, 4),
 ('Idaho', 2017, 3419, 4),
 ('Idaho', 2017, 2838, 51),
 ('Idaho', 2017, 3419, 51),
 ('Idaho', 2018, 3612, 4),
 ('Idaho', 2018, 3612, 51),
 ('Idaho', 2019, 4197, 4),
 ('Idaho', 2019, 4197, 51)]

In [80]:
data_collector.get_answers_for_q_less_occ(2, 10)

[('non-binary', 6)]

In [81]:
data_collector.get_answers_for_q_greater_occ(2, '10')

[('Female', 1024),
 ('Male', 3044),
 ('Non-binary', 13),
 ('Other', 107),
 ('n/a', 24)]

In [82]:
%%time
data_collector.get_different_answers_for_2q(3, 50)

Wall time: 41 s


[('United States of America', 1511),
 ('Spain', 1566),
 ('Canada', 1569),
 ('United Kingdom', 1595),
 ('United States of America', 1647),
 ('Canada', 1688),
 ('Canada', 1702),
 ('Netherlands', 1729),
 ('Lithuania', 1777),
 ('Algeria', 1858),
 ('Pakistan', 1983),
 ('Australia', 2045),
 ('United Kingdom', 2058),
 ('Other', 2081),
 ('Romania', 2095),
 ('Japan', 2152),
 ('France', 2179),
 ('Canada', 2288),
 ('France', 2335),
 ('Germany', 2359),
 ('Italy', 2440),
 ('Italy', 2449),
 ('Taiwan', 2626),
 ('Afghanistan', 2668),
 ('Canada', 2683),
 ('United States of America', 2690),
 ('Czech Republic', 2772),
 ('Belgium', 2789),
 ('Germany', 2880),
 ('Brazil', 2883),
 ('Hungary', 2884),
 ('Greece', 2912),
 ('Ireland', 2928),
 ('Switzerland', 2954),
 ('India', 3031),
 ('Argentina', 3074),
 ('Belarus', 3120),
 ('Netherlands', 3134),
 ('France', 3140),
 ('-1', 3447),
 ('-1', 3449),
 ('Poland', 3547),
 ('New Zealand', 3683),
 ('Belgium', 3705),
 ('Germany', 3736),
 ('France', 3760),
 ('Japan', 3807)

In [83]:
%%time
data_collector.get_different_answers_for_2q_v2(3, 50)

Wall time: 89 ms


[('Netherlands', 'United Kingdom', 1729),
 ('Canada', 'United States of America', 1702),
 ('Poland', 'United States of America', 3547),
 ('United States of America', 'Mexico', 3847),
 ('Romania', 'United States of America', 2095),
 ('Japan', 'Canada', 2152),
 ('Greece', 'Netherlands', 2912),
 ('India', 'United States of America', 3031),
 ('-1', 'n/a', 3449),
 ('France', 'United Kingdom', 2179),
 ('Spain', 'United States of America', 1566),
 ('Germany', 'United Kingdom', 2359),
 ('Italy', 'United States of America', 2440),
 ('Other', 'Germany', 2081),
 ('United Kingdom', 'Other', 3911),
 ('Canada', 'United States of America', 2683),
 ('Belarus', 'United States of America', 3120),
 ('Germany', 'Eritrea', 2880),
 ('Switzerland', 'Swaziland', 2954),
 ('Lithuania', 'United Kingdom', 1777),
 ('France', 'Switzerland', 3760),
 ('Belgium', 'Portugal', 2789),
 ('Taiwan', 'United States of America', 2626),
 ('Hungary', 'Austria', 2884),
 ('Algeria', 'United States of America', 1858),
 ('Netherlan

In [40]:
data_collector.get_all_answers_based_on_answer_and_q(2, 'Female')[:10]

[('37', 2014, 1, 1),
 ('Female', 2014, 1, 2),
 ('United States of America', 2014, 1, 3),
 ('Illinois', 2014, 1, 4),
 ('n/a', 2014, 1, 5),
 ('No', 2014, 1, 6),
 ('1', 2014, 1, 7),
 ('6-25', 2014, 1, 8),
 ('1', 2014, 1, 9),
 ('Yes', 2014, 1, 10)]

In [41]:
data_collector.get_some_answers_based_on_answer_and_q([2, 3, 4, 5], 2, 'Female')[:10]

[('Female', 2014, 1, 2),
 ('United States of America', 2014, 1, 3),
 ('Illinois', 2014, 1, 4),
 ('n/a', 2014, 1, 5),
 ('Female', 2014, 7, 2),
 ('United States of America', 2014, 7, 3),
 ('Michigan', 2014, 7, 4),
 ('n/a', 2014, 7, 5),
 ('Female', 2014, 9, 2),
 ('United States of America', 2014, 9, 3)]

In [73]:
data_collector.get_answers_distribution_for_q(7)

[('0', 42.82), ('1', 57.18)]

In [4]:
data_collector.get_answers_distribution_for_q(8)

[('1-5', 6.02),
 ('100-500', 18.68),
 ('26-100', 19.54),
 ('500-1000', 5.86),
 ('6-25', 16.33),
 ('More than 1000', 21.62),
 ('n/a', 11.95)]

In [25]:
data_collector.get_answers_distribution_for_q(54)

[('No', 0.88),
 ('Not applicable to me', 1.72),
 ('Unsure', 2.03),
 ('Yes', 12.41),
 ('n/a', 82.96)]

In [14]:
data_collector.get_avg_for_quantitative_q(1)

33.88

In [44]:
data_collector.get_hist_for_quantitative_q(1)

(array([ 735, 1749, 1139,  415,  125,   34,    5,    0,    0,    1],
       dtype=int64),
 array([18. , 26.1, 34.2, 42.3, 50.4, 58.5, 66.6, 74.7, 82.8, 90.9, 99. ]))

In [23]:
data_collector.get_users_no_per_answer_by_years(2)

[('Female', 2014, 247),
 ('Male', 2014, 991),
 ('Other', 2014, 21),
 ('non-binary', 2014, 1),
 ('Female', 2016, 336),
 ('Male', 2016, 1057),
 ('Non-binary', 2016, 2),
 ('Other', 2016, 31),
 ('n/a', 2016, 3),
 ('non-binary', 2016, 4),
 ('Female', 2017, 218),
 ('Male', 2017, 502),
 ('Non-binary', 2017, 6),
 ('Other', 2017, 17),
 ('n/a', 2017, 13),
 ('Female', 2018, 125),
 ('Male', 2018, 266),
 ('Non-binary', 2018, 1),
 ('Other', 2018, 21),
 ('n/a', 2018, 3),
 ('non-binary', 2018, 1),
 ('Female', 2019, 98),
 ('Male', 2019, 228),
 ('Non-binary', 2019, 4),
 ('Other', 2019, 17),
 ('n/a', 2019, 5)]

In [31]:
data_collector.get_answers_distribution_for_q_by_years(2)

[('Female', 2014, 19.6),
 ('Male', 2014, 78.65),
 ('Non-binary', 2014, 0.08),
 ('Other', 2014, 1.67),
 ('Female', 2016, 23.45),
 ('Male', 2016, 73.76),
 ('Non-binary', 2016, 0.42),
 ('Other', 2016, 2.16),
 ('n/a', 2016, 0.21),
 ('Female', 2017, 28.84),
 ('Male', 2017, 66.4),
 ('Non-binary', 2017, 0.79),
 ('Other', 2017, 2.25),
 ('n/a', 2017, 1.72),
 ('Female', 2018, 29.98),
 ('Male', 2018, 63.79),
 ('Non-binary', 2018, 0.48),
 ('Other', 2018, 5.04),
 ('n/a', 2018, 0.72),
 ('Female', 2019, 27.84),
 ('Male', 2019, 64.77),
 ('Non-binary', 2019, 1.14),
 ('Other', 2019, 4.83),
 ('n/a', 2019, 1.42)]